In [1]:
import sympy as sp
from sympy import init_printing
init_printing()
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent))
from cores.dynamical_systems.cartesian_arm_2_link import CartesianArmTwoLink
import torch
import numpy as np

In [2]:
m1 = 1.0
m2 = 1.0

In [3]:
system_torch = CartesianArmTwoLink(mass_link_1=m1, mass_link_2=m2, dtype=torch.float32)

In [4]:
d1, d2, d1_dot, d2_dot = sp.symbols(r'd_1 d_2 \dot{d}_1 \dot{d}_2')
u1, u2 = sp.symbols(r'u_1, u_2')
states = sp.Matrix([d1, d2, d1_dot, d2_dot])
controls = sp.Matrix([u1, u2])

In [5]:
f1 = d1_dot
f2 = d2_dot
f3 = u1/(m1+m2)
f4 = u2/m2
f = sp.Matrix([f1, f2, f3, f4])

# Test f(x,u)

In [7]:
f_func = sp.lambdify((states, controls), f)
state_np = np.array([1., 2., 3.0, 5.0], dtype=np.float32)
control_np = np.array([3., 1.0], dtype=np.float32)
print(f_func(state_np, control_np))


[[3. ]
 [5. ]
 [1.5]
 [1. ]]


In [8]:
state_torch = torch.tensor(state_np, dtype=torch.float32).unsqueeze(0)
control_torch = torch.tensor(control_np, dtype=torch.float32).unsqueeze(0)
f_torch = system_torch(state_torch, control_torch)
print(f_torch)

tensor([[3.0000, 5.0000, 1.5000, 1.0000]])


# Test df/dx

In [9]:
df_dx = f.jacobian(states)
df_dx_func = sp.lambdify((states, controls), df_dx)
print(df_dx_func(state_np, control_np))

[[0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]


In [10]:
df_dx_torch = system_torch.f_dx(state_torch, control_torch)
print(df_dx_torch)

tensor([[[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


# Test df/du

In [11]:
df_du = f.jacobian(controls)
df_du_func = sp.lambdify((states, controls), df_du)
print(df_du_func(state_np, control_np))

[[0.  0. ]
 [0.  0. ]
 [0.5 0. ]
 [0.  1. ]]


In [12]:
df_du_torch = system_torch.f_du(state_torch, control_torch)
print(df_du_torch)

tensor([[[0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.5000, 0.0000],
         [0.0000, 1.0000]]])


# d^2f/dx^2

In [13]:
f1_dxdx = f1.diff(states).jacobian(states)
print(f1_dxdx)

Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])


In [14]:
f2_dxdx = f2.diff(states).jacobian(states)
print(f2_dxdx)

Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])


In [15]:
f3_dxdx = f3.diff(states).jacobian(states)
print(f3_dxdx)

Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])


In [16]:
f4_dxdx = f4.diff(states).jacobian(states)
print(f4_dxdx)

Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
